In [1]:
import tensorflow as tf

In [2]:
def func():
    with tf.variable_scope('h1') as scope:
        w = tf.get_variable('w', [1])
        print w.name
        print scope.name
    with tf.variable_scope('h2') as scope:
        w = tf.get_variable('w', [2])
        print w.name
        print scope.name

In [3]:
with tf.variable_scope('func_call') as scope:
    func()
    scope.reuse_variables()
    func()

func_call/h1/w:0
func_call/h1
func_call/h2/w:0
func_call/h2
func_call/h1/w:0
func_call/h1
func_call/h2/w:0
func_call/h2


In [4]:
func()

h1/w:0
h1
h2/w:0
h2


In [8]:
import pandas as pd
import numpy as np

In [25]:
df = pd.DataFrame(np.arange(2400).reshape((600,4)))

In [26]:
def f1(df):
    df = df.iloc[np.random.permutation(len(df))]
    df = df.reset_index(drop=True)
    print df

In [27]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23
6,24,25,26,27
7,28,29,30,31
8,32,33,34,35
9,36,37,38,39


In [28]:
f1(df)

        0     1     2     3
0     556   557   558   559
1    1932  1933  1934  1935
2     588   589   590   591
3     684   685   686   687
4    1308  1309  1310  1311
5    2036  2037  2038  2039
6    1396  1397  1398  1399
7     724   725   726   727
8      52    53    54    55
9     472   473   474   475
10    460   461   462   463
11   1548  1549  1550  1551
12   2140  2141  2142  2143
13    688   689   690   691
14   2388  2389  2390  2391
15   2384  2385  2386  2387
16   2136  2137  2138  2139
17   2000  2001  2002  2003
18   1624  1625  1626  1627
19   2184  2185  2186  2187
20   1016  1017  1018  1019
21   2032  2033  2034  2035
22   1804  1805  1806  1807
23    816   817   818   819
24    252   253   254   255
25   2144  2145  2146  2147
26   1244  1245  1246  1247
27    488   489   490   491
28    940   941   942   943
29   1108  1109  1110  1111
..    ...   ...   ...   ...
570  2052  2053  2054  2055
571  2124  2125  2126  2127
572   796   797   798   799
573   740   741   74

In [24]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23
